#Lab.08 / IBM3202 – Análisis de Trayectorias utilizando MDanalysis

#Aspectos Teóricos

Ahora que ya has generado una trayectoria de dinámica molecular en el tutorial anterior, es crucial obtener percepciones cuantificables sobre tu sistema molecular. Hay un puñado de métricas que se pueden utilizar para lograr esto; aquí nos enfocaremos en dos de las más populares: **RMSD** y **RMSF**, así como distancias. Debido a limitaciones de tiempo, no vamos a cubrir métricas más avanzadas, pero estarán disponibles como un apéndice.


<figure>
<center>
<img src="https://amarolab.ucsd.edu/syncImages/c0b042e1-4fe9-4727-9c0c-f556edb1b4a7sars_cov2_spike_protein.gif"/>
<figcaption>FIGURA 1. Simulaciones de dinámica molecular (MD) de la proteína de espiga de SARS-CoV-2 glicosilada unida a una membrana. Tomado de <a href="https://amarolab.ucsd.edu">Amaro Lab</a> en UCSD.</figcaption></center>

</figure>

⚠️⚠️ La siguiente sección es un extracto adaptado de la introducción del tutorial de Cpptraj por Daniel R. Roe, disponible en [este enlace](http://ambermd.org/tutorials/analysis/tutorial1/).

## **Resumen del Desplazamiento Cuadrático Medio (RMSD)**

$RMSD$ mide la desviación de un conjunto objetivo de coordenadas (es decir, una estructura) respecto a un conjunto de coordenadas de referencia, siendo $RMSD=0$ indicativo de una superposición perfecta. En consecuencia, si tenemos una trayectoria de dinámica molecular (MD), se esperaría que cuanto menor sea el RMSD, menos cambios ocurran en la escala de tiempo estudiada.

$RMSD$ se define como:

<center>
<font size="5">
$RMSD = \sqrt{\frac{\sum_{i = 0}^N m_i(X_i - Y_i)^2}{M}}$
</font>
</center>

Donde **N** es el número de átomos, $m_i$ es la masa del átomo $i$, $X_i$ es el vector de coordenadas para el átomo objetivo $i$, $Y_i$ es el vector de coordenadas para el átomo de referencia $i$, y $M$ es la masa total. Si el $RMSD$ no está ponderado por la masa, para todos los $i$, $m_i = 1$, y $M = N$.
Al calcular el $RMSD$ de una estructura objetivo con respecto a una estructura de referencia, hay dos requisitos muy importantes, como veremos pronto en la parte práctica de este tutorial:

1. El número de átomos en la estructura objetivo debe coincidir con el número de átomos en la estructura de referencia.
2. El orden de los átomos en la estructura objetivo debe coincidir con el orden de los átomos en la estructura de referencia.


## **Resumen de la Fluctuación Cuadrática Media (RMSF)**


Como se menciona en la [guía del usuario de MDanalysis](https://userguide.mdanalysis.org/stable/examples/analysis/alignment_and_rms/rmsf.html):

> La fluctuación cuadrática media de la raíz ($RMSF$) de una estructura es **el promedio temporal del RMSD**. Se calcula según la siguiente ecuación, donde $x_i$
son las coordenadas de la partícula $i$ y $⟨x_i⟩$ es la posición promedio del conjunto de $i$:

<center>
<font size="5">
$ρ_i=\sqrt{⟨(x_i−⟨x_i⟩)^2⟩}$
</font>
</center>

> Mientras que el $RMSD$ cuantifica cuánto se desvía una estructura de una referencia a lo largo del tiempo, el **$RSMF$ puede revelar qué áreas del sistema son las más móviles**. Mientras que el $RMSD$ se calcula con frecuencia con respecto a un estado inicial, el $RMSF$ debe calcularse con respecto a una estructura promedio de la simulación. Un área de la estructura con valores altos de $RMSF$ a menudo se desvía de la media, lo que indica una alta movilidad. Cuando se realiza un análisis de $RMSF$ en proteínas, generalmente se restringe a los átomos del esqueleto o del carbono alfa; estos son más característicos de los cambios conformacionales que las cadenas laterales más flexibles.


<figure>
<center>
<img src='https://www.frontiersin.org/files/Articles/329304/fphar-09-00492-HTML/image_m/fphar-09-00492-g002.jpg'/>
<figcaption>FIGURA 2. Gráficos de RMSD y RMSF de los cambios estructurales que ocurren debido a la unión de un antagonista en el bolsillo de unión al ligando del receptor de andrógenos, elucidados a través de simulaciones de dinámica molecular.<br>Sugunadevi S et al (2018)<i> Front Pharmacology 9, 492</i> </figcaption></center>
</figure>

### Distancias
Como recordarás, los archivos de trayectoria almacenan la posición de cada átomo individual, por lo que el cálculo de distancias a través de una simulación de dinámica molecular suele ser bastante sencillo.



### Resumen del paquete MDAnalysis

Según se define en la [documentación](https://docs.mdanalysis.org/stable/documentation_pages/overview.html):

> **MDAnalysis** es un paquete de Python que proporciona clases para acceder a datos en trayectorias de dinámica molecular. Es orientado a objetos, por lo que trata átomos, grupos de átomos, trayectorias, etc., como objetos diferentes. Cada objeto tiene una serie de operaciones definidas en sí mismo (también conocidas como "métodos") y también contiene valores que describen el objeto ("atributos"). Por ejemplo, un objeto **AtomGroup** tiene un método **center_of_mass()** que devuelve el centro de masa del grupo de átomos. También contiene un atributo llamado residuos que enumera todos los residuos que pertenecen al grupo. Utilizando métodos como **select_atoms()** (que utiliza comandos de selección de átomos en estilo CHARMM), se pueden crear nuevos objetos (en este caso, otro **AtomGroup**).


**Ejemplo de código con MDAnalysis**

Un patrón de uso típico es iterar a través de una trayectoria y analizar las coordenadas para cada fotograma. En el siguiente ejemplo, se calcula la distancia de extremo a extremo de una proteína y el radio de giro de los átomos del esqueleto:




```
#!pip3 install --upgrade MDAnalysis
#!pip install --upgrade MDAnalysisTests
import MDAnalysis
from MDAnalysis.tests.datafiles import PSF,DCD  # test trajectory
import numpy.linalg
u = MDAnalysis.Universe(PSF,DCD)  # always start with a Universe
nterm = u.select_atoms('segid 4AKE and name N')[0]  # can access structure via segid (s4AKE) and atom name
cterm = u.select_atoms('segid 4AKE and name C')[-1]  # ... takes the last atom named 'C'
bb = u.select_atoms('protein and backbone')  # a selection (a AtomGroup)
for ts in u.trajectory:  # iterate through all frames
    r = cterm.pos - nterm.pos  # end-to-end vector from atom positions
    d = numpy.linalg.norm(r)   # end-to-end distance
    rgyr = bb.radius_of_gyration()  # method of a AtomGroup; updates with each frame
    print "frame = %d: d = %f Angstroem, Rgyr = %f Angstroem" % (ts.frame, d, rgyr)
```



## Conceptos básicos de MD analysis


1.   Universos y grupos de átomos
2.   Selecciones



**Universo y AtomGroup**

MDAnalysis es orientado a objetos. Los sistemas moleculares consisten en objetos `Átomo` (instancias de la clase `MDAnalysis.core.groups.Atom`), los cuales están agrupados en instancias de `AtomGroup`. Construyes el `AtomGroup` de tu sistema cargando una topología (una lista de átomos y posiblemente su conectividad) junto con una trayectoria (información de coordenadas) en la estructura de datos central, el objeto `Universo`:



```
u = MDAnalysis.Universe(PSF, DCD)
print(u)
<Universe with 3341 atoms>
```



**Selecciones**

MDAnalysis cuenta con una instalación de selección de átomos bastante completa. Principalmente, se utiliza el método select_atoms() de un Universe:




```
>>> CA = u.select_atoms("protein and name CA")
>>> CA
>>> <AtomGroup with 214 atoms>
```


Pero en realidad, cualquier AtomGroup tiene un método select_atoms():

```
>>> acidic = CA.select_atoms("resname ASP or resname GLU")
>>> acidic
<AtomGroup with 35 atoms>
>>> list(acidic.residues)
[<Residue GLU, 22>,
 <Residue ASP, 33>,
 <Residue GLU, 44>,
 ...
 <Residue GLU, 210>]
 ```

Revisa también todos los términos clave de selección se describen en la documentación.
Los rangos numéricos pueden escribirse como primero-último (o de manera equivalente, primero:último 1), donde el rango es inclusivo. Por ejemplo, obtener residuos con identificadores de residuo de 5 a 100:

```
>>> u.select_atoms("resid 5-100")
<AtomGroup with 1439 atoms>
>>> u.select_atoms("resid 5-100").n_residues
96
```
Las selecciones se pueden combinar con expresiones booleanas. Por ejemplo, para seleccionar los átomos de Cα de todos los residuos ácidos [ácido aspártico ("ASP"), ácido glutámico ("GLU") e histidinas (llamadas "HIS", "HSD" o "HSE", según el campo de fuerza que se esté utilizando y el estado de protonación)]:

```
>>> u.select_atoms("(resname ASP or resname GLU or resname HS*) and name CA")
<AtomGroup with 38 atoms>
```
Agrupamos o separamos selecciones por el nombre del residuo (palabra clave resname). Primero se seleccionan ASP, GLU o cualquier histidina (usamos "stemming" HS* para que coincida con cualquier nombre de residuo que comience con "HS"). Luego, solo se toman aquellos átomos cuyo nombre es "CA" del primer conjunto mediante una selección con "and". Para mayor comodidad, el "or" en la primera parte de la selección puede tomarse implícitamente con la sintaxis abreviada.

```
>>> u.select_atoms("resname ASP GLU HS* and name CA")
<AtomGroup with 38 atoms>
```


Si deseas profundizar en la sintaxis de selección de MDAnalysis, puedes leer la [documentación completa aquí](https://docs.mdanalysis.org/1.0.0/documentation_pages/selections.html).

# Aspectos Experimentales

Para este tutorial, vamos a utilizar una trayectoria de dinámica molecular (MD) del dominio de unión al ADN de la integrasa 1 del VIH. Como puedes ver en la entrada del PDB [aquí](https://www.rcsb.org/structure/1IHV), esta estructura fue resuelta utilizando resonancia magnética nuclear (NMR) y se determinó que forma un dímero en solución.

Aquí analizaremos 1000 frames de la integrasa en sus estados monomérico y dímero, y compararemos sus RMSD, RMSF y mediremos distancias.


<figure>
<center>
<img src='https://cdn.rcsb.org/images/structures/ih/1ihv/1ihv_chain-A.jpeg'/>
<figcaption>FIGURA 3. Representación en cintas de la Integrasa 1 del VIH (PDB 1IHV)</figcaption></center>
</figure>



# Parte 0: Descarga e Instalación del Software Requerido



## Instalación

Para realizar este tutorial, es necesario instalar los siguientes programas:
- **MDAnalysis** para analizar los datos en trayectorias de dinámica molecular.
- **py3Dmol** para la visualización de la estructura de la proteína.



In [ ]:
!pip3 install --upgrade MDAnalysis
# Importar MDAnalysis
import MDAnalysis as mda
#from MDAnalysis.tests.datafiles import PSF, DCD, DCD2
from MDAnalysis.analysis import gnm
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Instalación de py3Dmol mediante pip
!pip install py3Dmol
import py3Dmol

In [ ]:
!wget http://www.rcsb.org/pdb/files/1IHV.pdb.gz
!gunzip 1IHV.pdb.gz

In [ ]:
# Podemos visualizar la conformación dímera
import py3Dmol

view = py3Dmol.view()
view.addModel(open('1IHV.pdb', 'r').read(), 'pdb')

# Hacer zoom a todas las estructuras visualizadas
view.zoomTo()

# Establecer el color de fondo como blanco
view.setBackgroundColor('white')

# Establecer el estilo de visualización para las cadenas B y C
view.setStyle({'cartoon': {'color': 'purple'}})

# Finalmente, visualizamos las estructuras con el siguiente comando
view.show()


## Downloading MD trajectories

In [ ]:
# Aquí copiamos a nuestra instancia de Colab los archivos de trayectoria para el monómero y el dímero
!wget https://github.com/pb3lab/ibm3202/raw/master/files/md_files/1ihv_dimer_protPBC.xtc
!wget https://github.com/pb3lab/ibm3202/raw/master/files/md_files/1ihv_mon_protPBC.xtc
!wget https://github.com/pb3lab/ibm3202/raw/master/files/md_files/1ihv_mon_protPBC.gro
!wget https://github.com/pb3lab/ibm3202/raw/master/files/md_files/1ihv_dimer_protPBC.gro


# Parte I – Cálculo de $RMSD$ y $RMSF$


## I.1 - RMSD


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
**Nota:** Las siguientes celdas forman parte del tutorial de MDAnalysis disponible [aquí](https://userguide.mdanalysis.org/stable/examples/analysis/alignment_and_rms/aligning_trajectory_to_frame.html)
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



© Copyright 2019-2020, Lily Wang, Irfan Alibay, Rocco Meli, Mieczyslaw Torchala, Yuxuan Zhuang, Richard J. Gowers, and Oliver Beckstein.





### I.1A - Cálculo de $RMSD$ respecto a un marco de referencia


In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import align, rms

Primero necesitamos cargar nuestros archivos de trayectoria en MDAnalysis. Esto se hace creando una instancia de un objeto **Universe**.


In [ ]:
# Aquí creamos dos Universos, cada uno conteniendo la misma trayectoria de la trayectoria monomérica,
# uno llamado 'mobile' y el otro 'ref' que se utilizará como referencia
mobile = mda.Universe("/content/1ihv_mon_protPBC.gro", "/content/1ihv_mon_protPBC.xtc")
ref = mda.Universe("/content/1ihv_mon_protPBC.gro", "/content/1ihv_mon_protPBC.xtc")


Mientras que `align.alignto` alinea estructuras individuales o un marco de una trayectoria, `align.AlignTraj` alinea eficientemente una trayectoria completa con respecto a una referencia.

Primero verificamos el $RMSD$ de nuestra trayectoria no alineada para poder comparar resultados más tarde. El código a continuación establece la trayectoria `mobile` en el último marco indexando el último paso temporal, `ref` en el primer marco indexando el primer paso temporal, y calcula la desviación cuadrática media entre las posiciones de los átomos de carbono alfa ($\alpha$).


In [ ]:
mobile.trajectory[-1]  # establecer la trayectoria móvil en el último marco
ref.trajectory[0]  # establecer la trayectoria de referencia en el primer marco

mobile_ca = mobile.select_atoms('name CA')
ref_ca = ref.select_atoms('name CA')
rms.rmsd(mobile_ca.positions, ref_ca.positions, superposition=False)

Ahora podemos alinear la trayectoria. Ya hemos establecido 'ref' en el primer marco. En la celda siguiente, cargamos las posiciones de la trayectoria en memoria para que podamos modificar la trayectoria en Python.

In [ ]:
aligner = align.AlignTraj(mobile, ref, select='name CA', in_memory=True).run()

In [ ]:
mobile.trajectory[-1]  # establecer la trayectoria móvil en el último marco
ref.trajectory[0]  # establecer la trayectoria de referencia en el primer marco

mobile_ca = mobile.select_atoms('name CA')
ref_ca = ref.select_atoms('name CA')
rms.rmsd(mobile_ca.positions, ref_ca.positions, superposition=False)

**Pregunta❓:** Cómo cambia el RMSD antes y después del alineamiento?

### I.1B - RMSD de un Universo con múltiples selecciones a lo largo del tiempo


Es más eficiente utilizar la clase MDAnalysis.analysis.rms.RMSD para calcular el $RMSD$ de toda una trayectoria con respecto a un único punto de referencia.

La clase rms.RMSD primero realiza una alineación rotacional y translacional de la trayectoria objetivo con respecto al universo de referencia en `ref_frame`, utilizando los átomos en `select` para determinar la transformación. Luego, sin más alineación, se calcula el $RMSD$ de cada grupo en el argumento `groupselections`.

[Fuente](https://userguide.mdanalysis.org/stable/examples/analysis/alignment_and_rms/rmsd.html)


In [ ]:
# Aquí creamos dos Universos, cada uno conteniendo la misma trayectoria de la trayectoria monomérica,
# uno llamado 'monomer_mobile' y el otro 'monomer_ref' que se utilizará como referencia
monomer_mobile = mda.Universe("/content/1ihv_mon_protPBC.gro", "/content/1ihv_mon_protPBC.xtc")
monomer_ref = mda.Universe("/content/1ihv_mon_protPBC.gro", "/content/1ihv_mon_protPBC.xtc")

In [ ]:
rms.rmsd(monomer_mobile.select_atoms('backbone').positions,  # coordenadas a alinear
         monomer_ref.select_atoms('backbone').positions,  # coordenadas de referencia
         center=True,  # restar el centro de geometría
         superposition=True)  # superponer coordenadas

In [ ]:
# Aquí definimos dos
Loop1 = 'backbone and resid 227-240'
Loop2 = 'backbone and resid 252-257'

In [ ]:
# Aquí calculamos el RMSD
R_rmsd = rms.RMSD(mobile,  # universo a alinear
             ref,  # universo o grupo de átomos de referencia
             select='backbone',  # grupo para superponer y calcular el RMSD
             groupselections=[Loop1, Loop2],  # grupos para el RMSD
             ref_frame=0)  # índice del marco de referencia
R_rmsd.run()

Los datos se guardan en R_rmsd.results.rmsd como un array. Podemos verificar las dimensiones del array utilizando el atributo *shape*.


In [ ]:
R_rmsd.results.rmsd.shape

La variable `R_rmsd.results.rmsd` tiene una fila para cada paso temporal. Las dos primeras columnas de cada fila son el índice del marco del paso temporal y el tiempo (que se estima en formatos de trayectoria sin pasos temporales). La tercera columna es el $RMSD$ del argumento `select`. Las últimas columnas son el $RMSD$ de los grupos en `groupselections`.


#### Graficando los datos

Podemos visualizar fácilmente estos datos utilizando el paquete común de análisis de datos pandas. Convertimos el array `R_rmsd.results.rmsd` en un DataFrame y etiquetamos cada columna a continuación.


In [ ]:
import pandas as pd
df_rmsd_mono = pd.DataFrame(R_rmsd.results.rmsd,
                  columns=['Frame', 'Time (ns)','Backbone','Loop1','Loop2'])

df_rmsd_mono

Aquí utilizamos Plotly para crear fácilmente un gráfico interactivo.


In [ ]:
import plotly.graph_objects as go
import plotly.express as px
fig = px.line(df_rmsd_mono, x="Frame", y="Backbone",
        line_shape="spline", render_mode="svg",
        labels={ "Backbone": "RMSD(Å)" })
fig.add_scatter(x=df_rmsd_mono["Frame"], y=df_rmsd_mono["Loop1"], name="Loop 1", showlegend=True )
fig.add_scatter(x=df_rmsd_mono["Frame"], y=df_rmsd_mono["Loop2"], name="Loop 2")
fig.add_scatter(x=df_rmsd_mono["Frame"], y=df_rmsd_mono["Backbone"], name="Backbone" )
fig.show()

**Pregunta:** Cuál es el rango (en angstroms) de las fluctuaciones de RMSD?

### I.1B - Tu turno de calcular el RMSD del dímero

In [ ]:
# Aquí creamos dos Universos, cada uno conteniendo la misma trayectoria de la trayectoria dímera,
# uno llamado 'dimer_mobile' y el otro 'dimer_ref' que se utilizará como referencia
#dimer_mobile = mda.Universe("/content/1ihv_dimer_protPBC.gro", "/content/1ihv_dimer_protPBC.xtc")
#dimer_ref = mda.Universe("/content/1ihv_dimer_protPBC.gro", "/content/1ihv_dimer_protPBC.xtc")


In [ ]:
# rms.rmsd(dimer_mobile.select_atoms('backbone').positions,  # coordenadas a alinear
#          dimer_ref.select_atoms('backbone').positions,  # coordenadas de referencia
#          center=True,  # restar el centro de geometría
#          superposition=True)  # superponer coordenadas

In [ ]:
# Loop1A = 'backbone and resid 227-240'
# Loop2A = 'backbone and resid 252-257'

In [ ]:
# R_rmsd_dimer = rms.RMSD(dimer_mobile,  # universe to align
#              dimer_ref,  # reference universe or atomgroup
#              select='backbone',  # group to superimpose and calculate RMSD
#              groupselections=[Loop1A, Loop2A],  # groups for RMSD
#              ref_frame=0)  # frame index of the reference
# R_rmsd_dimer.run()

Los datos se guardan en `R_rmsd.results.rmsd` como un array. Podemos verificar las dimensiones del array utilizando el atributo `shape`.


In [ ]:
R_rmsd_dimer.results.rmsd.shape

In [ ]:
import pandas as pd
# Aquí creamos el DataFrame de pandas a partir del objeto R_rmsd_dimer.rmsd
df_rmsd_dimer = pd.DataFrame(R_rmsd_dimer.results.rmsd,
                  columns=['Frame', 'Time (ns)','Backbone','Loop1A','Loop2A'])

df_rmsd_dimer

Veamos la evolución del $RMSD$ para el mónomero y el dímero

In [ ]:
import plotly.express as px

fig = px.line(df_rmsd_mono, x="Frame", y="Backbone",
        line_shape="spline", render_mode="svg",
        labels={ "Backbone": "RMSD(Å)" })
fig.add_scatter(x=df_rmsd_mono["Frame"], y=df_rmsd_mono["Backbone"], name="Backbone Monomer" )
fig.add_scatter(x=df_rmsd_dimer["Frame"], y=df_rmsd_dimer["Backbone"], name="Backbone Dimer AVG" )
fig.show()

Ahora es tu turno de explorar si hay algún cambio en el RMSD de los loops.

In [ ]:
#Hints:
fig.add_scatter(x=df_rmsd_mono["Frame"], y=df_rmsd_mono["Loop1"], name="Loop 1", showlegend=True )
fig.add_scatter(x=df_rmsd_mono["Frame"], y=df_rmsd_mono["Loop2"], name="Loop 2")

##I.2 - RMSF

Ahora queremos evaluar las fluctuaciones atómicas promedio durante las trayectorias de MD para ambos estados monomérico y dímero de la integrasa.


In [ ]:
# Primero necesitamos asegurarnos de que nuestros universos estén correctamente alineados

aligner = align.AlignTraj(monomer_mobile, monomer_ref, select='name CA', in_memory=True).run()
aligner = align.AlignTraj(dimer_mobile, dimer_ref, select='name CA', in_memory=True).run()

In [ ]:
# Aquí creamos una selección de la trayectoria previamente alineada
c_alphas_monomer = monomer_mobile.select_atoms('protein and name CA')
c_alphas_dimer = dimer_mobile.select_atoms('protein and name CA')
R_rmsf_mono = rms.RMSF(c_alphas_monomer).run()
R_rmsf_dimer = rms.RMSF(c_alphas_dimer).run()
rms.RMSF

In [ ]:
c_alphas_dimer.resids

In [ ]:
import pandas as pd
#creamos un df de pandas
df_rmsf_mono = pd.DataFrame(R_rmsf_mono.results.rmsf,
                  columns=['BackboneRMSF'])
df_rmsf_mono = df_rmsf_mono.assign(Residue = c_alphas_monomer.resids)

df_rmsf_dimer = pd.DataFrame(R_rmsf_dimer.results.rmsf,
                  columns=['BackboneRMSF'])
df_rmsf_dimer = df_rmsf_dimer.assign(Residue = c_alphas_dimer.resids)


df_rmsf_dimer_A = df_rmsf_dimer.head(52)
df_rmsf_dimer_B = df_rmsf_dimer.tail(52)



In [ ]:
import plotly.express as px

fig = px.line(df_rmsf_mono, x="Residue", y="BackboneRMSF",
        line_shape="linear", render_mode="svg",
        labels={ "BackboneRMSF": "RMSF(Å)" , "Residue":"Residue Number"}, color=None)
fig.add_scatter(x=df_rmsf_dimer_A["Residue"], y=df_rmsf_dimer_A["BackboneRMSF"], name="Dimer Chain A", line_shape="linear")
fig.add_scatter(x=df_rmsf_dimer_B["Residue"], y=df_rmsf_dimer_B["BackboneRMSF"], name="Dimer Chain B", line_shape="linear")
fig.add_scatter(x=df_rmsf_mono["Residue"], y=df_rmsf_mono["BackboneRMSF"], name="Monomer", line_shape="linear")

fig.show()


**Preguntas❓**

1. Hay alguna diferencia entre las fluctuaciones locales entre monómero y dímero?

2. Qué región exhibe una mayor fluctuación atómica a través de la trayectoria?

3. Cuáles son las características estructurales de estas regiones?



In [ ]:
#visualiza el estado monomérico

## I.3 -  RMSD de a pares

In [ ]:
#Import modules
import MDAnalysis as mda
from MDAnalysis.analysis import diffusionmap, align

Los RMSDs par a par son una forma efectiva de ver rápidamente similitudes y diferencias en las conformaciones (medidas por RMSD) a lo largo de toda una trayectoria y no solo en comparación con un único marco de referencia.

Vamos a utilizar las trayectorias previamente alineadas **monomer_mobile** y **dimer_mobile**.


Luego podemos calcular una matriz de $RMSD$ par a par con la clase `diffusionmap.DistanceMatrix`, utilizando la métrica rms.rmsd por defecto.


In [ ]:
# Cálculo de la matriz de distancias para el monómero
matrix1 = diffusionmap.DistanceMatrix(monomer_mobile, select='name CA').run()
# Cálculo de la matriz de distancias para el dímero
matrix2 = diffusionmap.DistanceMatrix(dimer_mobile, select='name CA').run()

El array de resultados está en `matrix.results.dist_matrix` como un array cuadrado con la forma (#n_frames, #n_frame).


In [ ]:
print(matrix1.results.dist_matrix.shape)
print(matrix2.results.dist_matrix.shape)

Podemos utilizar el paquete común de visualización matplotlib para crear un mapa de calor a partir de este array.


In [ ]:
# Aquí representamos gráficamente la matriz de RMSD para el monómero
plt.imshow(matrix1.results.dist_matrix, cmap='viridis')
plt.xlabel('Frame')
plt.ylabel('Frame')
plt.colorbar(label='RMSD (Angstrom)')


In [ ]:
# Aquí representamos gráficamente la matriz de RMSD para el dímero
plt.imshow(matrix2.results.dist_matrix, cmap='viridis')
plt.xlabel('Frame')
plt.ylabel('Frame')
plt.colorbar(label='RMSD (Angstrom)')

#Appendix A - Normal Mode y Principal Component Analysis  

## I - Análisis de Modos Normales y Contactos a Larga Distancia


In [ ]:
monomer_mobile = mda.Universe("/content/1ihv_mon_protPBC.gro", "/content/1ihv_mon_protPBC.xtc")
dimer_mobile = mda.Universe("/content/1ihv_dimer_protPBC.gro", "/content/1ihv_dimer_protPBC.xtc")

In [ ]:
nma1 = gnm.GNMAnalysis(monomer_mobile, select='protein and name CA', cutoff=7.0)
nma1.run()

In [ ]:
nma2 = gnm.GNMAnalysis(dimer_mobile,
                      select='protein and name CA',
                      cutoff=7.0)
nma2.run()


In [ ]:
len(nma2.results)

In [ ]:
%matplotlib inline
#sns.set_context('notebook')
%config InlineBackend.figure_format = 'retina'

## visualizamos la distribución de los eigenvalues. la conformación dominante es represantada por un pico en:
eigenvalues1 = [res[1] for res in nma1.results]
eigenvalues2 = [res[1] for res in nma2.results]

histfig, histax = plt.subplots(nrows=2, sharex=True, sharey=True)
histax[0].hist(eigenvalues1)
histax[1].hist(eigenvalues2)

histax[1].set_xlabel('Eigenvalue')
histax[0].set_ylabel('Frequency (Monomer)')
histax[1].set_ylabel('Frequency (Dimer)');
plt.show()


In [ ]:
import pandas as pd
import numpy as np

##Create Panda Dataframe Files
eu = pd.DataFrame({'LC': eigenvalues1,'NLC': eigenvalues2})

#Save Panda DataFrame
eu.to_csv('./DF_1.csv')


#inspect Dataframe
eu.head()

In [ ]:
time1 = [res[0] for res in nma1.results]
time2 = [res[0] for res in nma2.results]
linefig, lineax = plt.subplots()
plt.plot(time1, eigenvalues1, label='Monomer')
plt.plot(time2, eigenvalues2, label='Dimer')
lineax.set_xlabel('Time (ps)')
lineax.set_ylabel('Eigenvalue')
plt.legend();
plt.show()

## II -PCA

In [ ]:
#Importar
import MDAnalysis as mda
from MDAnalysis.analysis import diffusionmap, align
import matplotlib.pyplot as plt
%matplotlib inline

**¡ADVERTENCIA!**

Para obtener mejores resultados, tu trayectoria debe estar alineada en tu selección de grupo de átomos antes de ejecutar el análisis. Configurar align=True no dará resultados correctos en el PCA.


In [ ]:
# Alinear
aligner1 = align.AlignTraj(monomer_mobile, monomer_mobile, select='backbone', in_memory=True).run()
aligner2 = align.AlignTraj(dimer_mobile, dimer_mobile, select='backbone', in_memory=True).run()


### Descripción general del método
El **análisis de componentes principales (PCA, por sus siglas en inglés)** es una técnica estadística que descompone un sistema de observaciones en variables linealmente no correlacionadas llamadas componentes principales. Estas componentes se ordenan de manera que la primera componente principal explica la mayor varianza en los datos, y cada componente siguiente explica una varianza cada vez menor. El PCA se aplica a menudo a trayectorias de dinámica molecular para **extraer las grandes movimientos conformacionales o "dineámicas esenciales" de una proteína**. La fluctuación conformacional paso a paso se puede considerar una combinación lineal de las dinámicas esenciales obtenidas mediante el PCA.

En MDAnalysis, el método es el siguiente:

> Opcionalmente, alinea cada marco de tu trayectoria con el primer marco.
Construye una matriz de covarianza de 3N x 3N para los N átomos en tu trayectoria. Opcionalmente, puedes proporcionar una media; de lo contrario, la covarianza es con respecto a la estructura promediada a lo largo de la trayectoria.
Diagonaliza la matriz de covarianza. Los eigenvectores son las componentes principales, y sus eigenvalores son la varianza asociada.
Ordena los eigenvalores de manera que las componentes principales estén ordenadas por varianza.


In [ ]:
import MDAnalysis as mda
import MDAnalysis.analysis.pca as pca
from MDAnalysis.coordinates.base import Timestep

import numpy as np
import os
import glob

import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.cm
import matplotlib.ticker as ticker
%matplotlib inline

### Llama a la función PCA
Puedes elegir cuántas componentes principales guardar en el análisis con `n_components`. El valor predeterminado es `None`, que guarda todas ellas. También puedes pasar una estructura de referencia media que se utilizará para calcular la matriz de covarianza. Con el valor predeterminado de `None`, la covarianza utiliza las coordenadas medias de la trayectoria.


In [ ]:
pcu1 = pca.PCA(monomer_mobile, select='protein and backbone',
             align=False, mean=None,
             n_components=None).run()
pcu2 = pca.PCA(dimer_mobile, select='protein and backbone',
             align=False, mean=None,
             n_components=None).run()


### Las componentes principales se guardan en `pc.p_components`.
Si guardaste todas las componentes, deberías tener un array de forma (natoms×3, natoms×3).



In [ ]:
backbone1 = monomer_mobile.select_atoms('protein and backbone')
n_bb1 = len(backbone1)
print('There are {} backbone atoms in the analysis'.format(n_bb1))
print(pcu1.p_components.shape)

backbone2 = dimer_mobile.select_atoms('protein and backbone')
n_bb2 = len(backbone2)
print('There are {} backbone atoms in the analysis'.format(n_bb2))
print(pcu2.p_components.shape)

### Varianza del primer componente

In [ ]:
pcu1.variance[0],pcu2.variance[0]

Esta varianza es algo insignificante por sí sola. Es mucho más intuitivo considerar la varianza de una componente principal como un porcentaje de la varianza total en los datos. MDAnalysis también realiza un seguimiento de la varianza acumulada en porcentaje en `pc.cumulated_variance`. Como se muestra a continuación, la primera componente principal contiene el 90.3% de la varianza total de la trayectoria. Las tres primeras componentes combinadas representan el 96.4% de la varianza total.


In [ ]:
print(pcu1.cumulated_variance[0])
print(pcu1.cumulated_variance[2])
print(pcu2.cumulated_variance[0])
print(pcu2.cumulated_variance[2])

In [ ]:
plt.plot(pcu1.cumulated_variance[:10])
plt.xlabel('Principal component')
plt.ylabel('Cumulative variance');

plt.plot(pcu2.cumulated_variance[:10])
plt.xlabel('Principal component')
plt.ylabel('Cumulative variance');

### Visualización de proyecciones en un espacio dimensional reducido


El método `pc.transform()` transforma un grupo de átomos dado en pesos $w_i$ sobre cada componente principal $i$.

$w_i(t) = (r(t) - \bar{r}) \cdot u_i$

$r(t)$ son las coordenadas del grupo de átomos en el tiempo $t$, $\bar{r}$ son las coordenadas medias utilizadas en el PCA, y $u_i$ es el vector propio de la componente principal $i$.

Si bien el grupo de átomos dado debe tener el mismo número de átomos sobre los que se calcularon las componentes principales, no tiene que ser el mismo grupo.

Nuevamente, pasar `n_components=None` transformará tu grupo de átomos sobre cada componente. A continuación, limitamos la salida a proyecciones sobre solo 5 componentes principales.


In [ ]:
transformed1 = pcu1.transform(backbone1, n_components=5)


transformed2 = pcu2.transform(backbone2, n_components=5)
transformed1.shape, transformed2.shape

La salida tiene la forma (n_frames, n_components). Para un análisis y representación más sencillos, podemos convertir el array en un DataFrame.


In [ ]:
df1 = pd.DataFrame(transformed1,
                  columns=['PC{}'.format(i+1) for i in range(5)])
df1['Time (ns)'] = df1.index * monomer_mobile.trajectory.dt
df1.head()

df2 = pd.DataFrame(transformed2,
                  columns=['PC{}'.format(i+1) for i in range(5)])
df2['Time (ns)'] = df2.index * dimer_mobile.trajectory.dt
df2.head()

Hay varias formas en las que podemos visualizar los datos. Utilizar la herramienta PairGrid de Seaborn es la forma más rápida y sencilla, si ya tienes seaborn instalado.


In [ ]:
import seaborn as sns

g1 = sns.PairGrid(df1, hue='Time (ns)',
                 palette=sns.color_palette('Oranges_d',
                                           n_colors=len(df1)))
g1.map(plt.scatter, marker='.')

In [ ]:
g2 = sns.PairGrid(df2, hue='Time (ns)',
                 palette=sns.color_palette('Oranges_d',
                                           n_colors=len(df2)))
g2.map(plt.scatter, marker='.')

Otra forma de investigar los movimientos esenciales de la trayectoria es proyectar la trayectoria original sobre cada una de las componentes principales para visualizar el movimiento de la componente principal. El producto de los pesos $w_i(t)$ para la componente principal $i$ con el vector propio $u_i$ describe las fluctuaciones alrededor de la media en ese eje, por lo que la trayectoria proyectada $r_i(t)$ es simplemente las fluctuaciones sumadas a las posiciones medias $\bar{r}$.

$r_i(t) = w_i(t) \times u_i + \bar{r}$
A continuación, generamos las coordenadas proyectadas de la primera componente principal. Las posiciones medias se almacenan en `pc.mean`.


In [ ]:
pc1u1 = pcu1.p_components[:, 0]
trans1u1 = transformed1[:, 0]
projectedu1 = np.outer(trans1u1, pc1u1) + pcu1.mean
coordinatesu1 = projectedu1.reshape(len(trans1u1), -1, 3)

pc1u2 = pcu2.p_components[:, 0]
trans1u2 = transformed2[:, 0]
projectedu2 = np.outer(trans1u2, pc1u2) + pcu2.mean
coordinatesu2 = projectedu2.reshape(len(trans1u2), -1, 3)

Podemos crear un nuevo universo a partir de esto para visualizar el movimiento sobre la primera componente principal.


In [ ]:
!pip3 install nglview
import nglview as nv

In [ ]:
proj1 = mda.Merge(backbone1)
proj1.load_new(coordinatesu1, order="fac")

In [ ]:
view = nv.show_mdanalysis(proj1.atoms)
view

Si tienes nglview instalado, puedes ver la trayectoria en el cuaderno. De lo contrario, puedes escribir la trayectoria en un archivo y usar otro programa como VMD. A continuación, creamos una película de la componente.


In [ ]:
!pip install moviepy==0.2.2.11
!pip install imageio==1.6

In [ ]:
from nglview.contrib.movie import MovieMaker
movie = MovieMaker(view, output='pc1u1.gif', in_memory=True)
movie.make()